In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random

import xgboost as xgb

#plot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib.axes import Axes

In [2]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_x_resample3000_boundarysel_50pergroup.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_y_resample3000_boundarysel_50pergroup.csv")
X_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\scaled_validation_feature.csv")
y_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\validation_label.csv")

In [3]:
X_train = X_train.drop(["predict_cluster"],axis=1)
y_train = y_train.drop(["predict_cluster"],axis=1)
X_val = X_val.drop(["Unnamed: 0"],axis=1)
y_val = y_val.drop(["Unnamed: 0"],axis=1)

In [4]:
def cal_score (y_pred,y_val):
    n11 = 0
    n12 = 0
    n21 = 0
    n22 = 0
    y_pred_array= np.array(y_pred)
    y_val_array= np.array(y_val)
    for j in range(len(y_pred_array)):
        if (y_pred_array[j]==2)&(y_val_array[j]==2):
            n22 = n22+1
        elif (y_pred_array[j]==1)&(y_val_array[j]==2):
            n12 = n12 +1
        elif (y_pred_array[j]==2)&(y_val_array[j]==1):
            n21 = n21+1
        else:
            n11 = n11+1   
    '''
    ita = np.nditer (y_pred_array)
    itb = np.nditer (y_val_array)
    while not ita.finished:
        if (ita[0]==2)&(itb[0]==2):
            n22 = n22+1
        elif (ita[0]==1)&(itb[0]==2):
            n12 = n12+1
        elif (ita[0]==2)&(itb[0]==1):
            n21 = n21+1
        else:
            n11 = n11+1
        ita.iternext()
        itb.iternext()
    '''
    try:       
        Precall = n22 / ( n12 + n22)
        Pprecision = n22 / ( n21 + n22)
        f1_score = 2 / (1/Precall + 1/Pprecision)
        FP = n21/(n21 + n11)
        FN = n12/(n12 +n22 )
        BER = 1/2*(FP+FN)   
    except Exception as ex:
        print (ex.message)
    print ("n11:.."+str(n11)+"..n12:.."+str(n12)+"..n21:.."+str(n21)+"..n22:.."+str(n22))
    print ("TP:"+str(Precall))
    print ("f1 score:" + str(f1_score))
    print ("FP:"+ str(FP))
    print ("BER:" + str(BER))
    return Precall,f1_score,BER,FP

In [6]:
xgb_clf=xgb.XGBClassifier(  n_estimators=500,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=0.03, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3
                     )

In [9]:
xgb_clf.fit(X_train,y_train['label'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=10, learning_rate=0.03,
       max_delta_step=0, max_depth=4, min_child_weight=6, missing=None,
       n_estimators=500, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=8,
       reg_lambda=1.3, scale_pos_weight=1.6, seed=None, silent=True,
       subsample=0.8)

In [10]:
y_pred =xgb_clf.predict(X_val)

In [11]:
y_pred = {"label_pred":y_pred}
y_pred = pd.DataFrame(data=y_pred)

In [13]:
Precall,f1_score,BER,FP = cal_score (y_pred,y_val['label']) 

n11:..55695..n12:..5136..n21:..5512..n22:..3684
TP:0.41768707482993195
f1 score:0.4089698046181172
FP:0.09005505906187201
BER:0.33618399211597005


In [14]:
xgb_clf.feature_importances_

array([ 0.16555925,  0.22683647,  0.22757475,  0.213732  ,  0.16629753], dtype=float32)